In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate as integrate
import scipy.io as sio
#from Lorenz96_RK4 import Lorenz96
from L96_Model import L96
import sys, os, time


In [23]:
# Data, Data Assimilation Hyperparameters
D = 20
Dobs = 12
dims = set(np.arange(20))
#dim_obs = [1, 2, 3, 5, 7, 9, 11, 13, 15, 17, 18, 19]
dim_obs = [0, 1, 2, 4, 6, 8, 10, 12, 14, 16, 17, 18]
dim_unobs = list(dims - set(dim_obs))
M = 200

# Annealing Hyperparameters
Rm = 1
Rf0 = 1e6
alpha = 1.3
betamax = 1


# Hamiltonian Monte Carlo Hyperparameters
niter = int(1e3)
Te = np.exp(1e-1*np.arange(niter))  # Temperature
epsilon = 1e-3*np.ones((1,niter))
L = 150
mass = [1e3, 1e-2, 1e0]
# Hamiltonian Monte Carlo Tuning Parameters
mass_X = np.zeros(shape=(D,M))
mass_X[dim_obs,:] = mass[0]
mass_X[dim_unobs,:] = mass[1]
mass_nu = mass[2]

mass_X_sqrt = np.zeros(shape=(D,M))
mass_X_sqrt[dim_obs,:] = np.sqrt(2*mass[0])
mass_X_sqrt[dim_unobs,:] = np.sqrt(2*mass[1])
mass_nu_sqrt = np.sqrt(2*mass[2])

# Post-processing
plot_Action_vs_beta = False
savedata = False

In [15]:
# Load the data set
gen_nu = "8.17"
gen_noise = "sig0.4"
gen_dt = "0.001"
gen_delta_t = "0.025"
gen_integrator = "RK4"

# Specify the data path
datapath = ("./L96_D%s_nu%s_%s_dt%s_deltat%s_%s.mat" % 
    (str(D), gen_nu, gen_noise, gen_dt, gen_delta_t, gen_integrator))

if datapath[-3:] == "mat":
    datadict = sio.loadmat(datapath)
    data = datadict["Data"]
    dt = datadict["delta_t"][0]
elif datapath[-3:] == "npy":
    data = np.load(datapath)
else:
    raise IOError ("Does not recognize data file extension\n datapath = %s" % datapath)
    sys.exit()

In [21]:
# Prepare the data
Y = data[dim_obs,:M]

#Initialize the state variables
nu_init = 8

np.random.seed(12345)
X_init = np.zeros((D,M))
X_init[:,0] = 20*np.random.random(size=(D))
X_init[dim_obs, :] = Y

for k in range(0, M-1): # should be M -1??
    X_init[:,k+1] = X_init[:,k] + dt *  L96(X_init[:,k]+dt/2*L96(X_init[:,k], nu_init),nu_init)
    X_init[dim_obs,k+1] = Y[:,k+1]

In [51]:
# Initilize vectorized dirac delta functions
eyeDleft1 = np.roll(np.eye(D), -1, 1)
eyeDleft2 = np.roll(np.eye(D),-2,1)
eyeDright = np.roll(np.eye(D),1,1)

# Some initializations for HMC

# Define the Rf ladder
Rf = Rf0 * (alpha**(np.arange(0,betamax)))

# Initialize the solutions
X_sol = X_init
nu_sol = nu_init
# Initialize the final output cell array. (It is a dictionary in order to replicate the cells data type in MATLAB)
q_min = {'X_sol': X_sol, 'nu_sol': nu_sol}

#Initialize action matrix
Action = np.zeros(shape=(betamax, niter))  #in MATLAB code, the shape is (betamax, niter+1). Not sure why the +1
Action_min = np.zeros(shape=(betamax,1))

# Percentage acceptance and percentage downhill
Acceptance = np.zeros(shape=(betamax,1))
Downhill = np.zeros(shape=(betamax, 1))

# Initialize Momentum
pX0 = np.zeros(shape=(D,M))

In [54]:
# Hamiltonian Monte Carlo Algorithm

for beta in range(1, betamax+1):
    # Initialize states (i.e. take the results from the previous step)
    X0 = X_sol
    X0(dim_obs,:) = Y  # Is this really necessary?
    nu0 = nu_sol
    
    # Evaluate the starting action under current beta
    Xup1, Xdown2, Xleft1, Zup1, Zdown2, Zdown1, hX = fun_getPieces(X, nu, dt)
    Action[beta-1, 1] = fun_A(X0, Xleft1, hX, Y, dim_obs, M, Rm, Rf[beta-1])
    Action_min[beta-1] = Action[beta-1, 1]
    
    print("Start annealing for beta = %d ..." % beta)
    start_time = time.time()
    
    for n in range(2, niter+1):
        eps = epsilon[n-1]

1


In [ ]:
def fun_getPieces(X, nu, dt):
    Xup1 = np.roll(X, -1, 0)
    Xdown1 = np.roll(X, 1, 0)
    Xdown2 = np.roll(X, 2, 0)
    Xleft1 = np.roll(X, -1, 1)
    
    Z = X + dt/2* (np.multiply(Xup1 - Xdown2, Xdown1) - X + nu)
    Zup1 = np.roll(Z, -1, 0)
    Zdown1 = np.roll(Z, 1, 0)
    Zdown2 = np.roll(Z, 2, 0)
    
    hX = X + dt * (np.multiply(Zup1 - Zdown2, Zdown1) - Z + nu)
    
    return Xup1, Xdown2, Xleft1, Zup1, Zdown2, Zdown1, hX

In [60]:
def fun_A(X0, Xleft1, hX, Y, dim_obs, M, Rm, Rf):
    kern1 = Rm/(2*M) * np.sum((X[dim_obs,:] - Y)**2)
    
    kern2 = Xleft1 - hX
    kern2 = Rf/(2*M) * np.sum(kern2[:,:M-1]**2) # Is this right? I feel like it's leaving out one column
    
    return kern1 + kern2

In [57]:
testarray = np.arange(9).reshape((3,3))
display(testarray)

array([[0, 1, 2],
       [3, 4, 5],
       [6, 7, 8]])

204

In [ ]:
def generate_data():
    """
        This function is optional, in case you do not want to use previous data
    """
    # Set the hyperparamters for generating the data
    # stepsize = dt
    stepsize = 0.01

    # How many time points
    steps = 1000

    # Initial point
    np.random.seed(123456)
    D = 20
    y0 = np.random.random(D)*3
    t0 = 0
    F = 8

    sample = Lorenz96(y0, t0, D, F)
    data = sample.generate(stepsize=stepsize, steps=steps, addNoise=True, noise_level=0.04)
    return data